In [2]:
# import
import pandas as pd

from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm


from datetime import datetime, timedelta
import asyncio
import json
import nest_asyncio
import re

nest_asyncio.apply()

In [3]:
#웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(3)

[WDM] - Downloading: 100%|██████████| 8.06M/8.06M [00:00<00:00, 17.1MB/s]
/var/folders/ks/zdzpzdss37j5m0f27_d5d7qh0000gn/T/ipykernel_38106/4096312416.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [98]:
driver.get('https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100&page=2#&date=%2000:00:00&page=1')
page_news = driver.find_element(By.CLASS_NAME, 'section_body').find_elements(By.TAG_NAME,'ul')

In [132]:
urls = []

prev = 'None'
i = 1
while True:
    if i == 200:
        break
    url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=100&page=2#&date=%2000:00:00&page={i}'
    driver.get(url)
    page_news = driver.find_element(By.CLASS_NAME, 'section_body').find_elements(By.TAG_NAME,'ul')
    driver.implicitly_wait(3)
    for chunck in page_news:
        try:
            for single_article in chunck.find_elements(By.TAG_NAME, 'li'):
                try:
                    urls.append(single_article.find_elements(By.TAG_NAME, 'a')[0].get_attribute('href'))
                except:
                    pass
        except:
            pass

    i += 1

In [137]:
urls_unique = list(set(urls))

In [138]:
urls_unique[0]

'https://n.news.naver.com/mnews/article/366/0000901090?sid=100'

In [139]:
def get_comments(refer_url, comment_url) : # 댓글 목록을 json 형태로 받아오는 함수
    comments = []
    next = None
    # 처음엔 댓글 개수를 모르므로 충분히 큰 수를 넣어 줌
    comment_count = 10e6
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Safari/605.1.15',
        'referer': refer_url
    }
    
    # 수집한 댓글 수가 첫번째에 수집한 총 댓글 수 보다 많다면 반복을 종료합니다.
    while len(comments) < comment_count :
        comment_url_next = comment_url + '&moreParam.next=' + next if next else comment_url
        res = requests.get(comment_url_next, headers=headers)
        dic = json.loads(res.text[res.text.index('(')+1:-2])
        comments.extend(list(map(lambda x : {
            'url' : refer_url,
            'text': x['contents'],
            'reply_count' : x['replyCount'], 
            'uid': x['idNo'],
            'uname' : x['userName'],
            'like': x['sympathyCount'], 
            'dislike': x['antipathyCount'],
            'c_time' : x['modTime'],
            'cid': x['commentNo'],
            'pid' : x['parentCommentNo'] 
            }, dic['result']['commentList'])))
        comment_count = dic['result']['count']['comment']
        next = dic['result']['morePage']['next'] if comment_count else None
    # 필터로 삭제된 댓글을 걸러줍니다
    comments=list(filter(lambda x: len(x['text']), comments))
    return comments


def get_data(oid, aid) :
    try :
        # refer_url: 댓글 보기를 누르면 나오는 댓글 페이지 주소
        # comment_url: 네트워크 탭에서 확인 가능한 동적으로 생성되는 주소
        refer_url = f'https://n.news.naver.com/mnews/article/comment/{oid}/{aid}?sid=100'
        comment_url = f'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics_m3&pool=cbox5&_cv=20220623185938&_callback=jQuery33103200637113167857_1656171100524&lang=ko&country=KR&objectId=news{oid}%2C{aid}&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=20&followSize=100&sort=new&includeAllStatus=true&_=1656171100525'

        comments = get_comments(refer_url, comment_url)
        comments = pd.DataFrame(comments)
        # 댓글 수가 0개인 기사를 어떻게 할지 추후에 결정해야 함
        return comments
    except :
        return None


In [140]:
df = pd.DataFrame()
for url in tqdm(urls_unique):
    try:
        oid = re.findall('article/([^\*]*)/', str(url))[0]
        aid = re.findall(f'{oid}/([^\*]*)[?]', str(url))[0]
        sid = url[-3:]
        temp = get_data(oid, aid)
        temp['sid'] = sid
        df =pd.concat([df, temp])
    except:
        print(url)

 49%|████▉     | 490/996 [04:15<09:36,  1.14s/it] 

In [148]:
now = datetime.now().date()
str(now)

'2023-05-19'

In [149]:
df.reset_index(drop=True, inplace=True)
df['date'] = datetime.now().date()
df

,url,text,reply_count,uid,uname,like,dislike,c_time,cid,pid,sid,date
0,https://n.news.naver.com/mnews/article/comment...,이런것이 법사위원을 맡고 있었으니... 참 치졸하고 한심하구나... 김남국! 이 쓰...,0.0,2Ub31,6666****,0.0,0.0,2023-05-18T09:27:51+0900,792083099679719776,792083099679719776,100,2023-05-19
1,https://n.news.naver.com/mnews/article/comment...,나 좌익 빨갱이라고 외치는 연예인 부터 퇴출시켜야,0.0,1tTcz,golf****,0.0,0.0,2023-05-13T06:35:58+0900,791608169712320687,791608169712320687,100,2023-05-19
2,https://n.news.naver.com/mnews/article/comment...,국회의원 박탈하고그동안 받은 월급 몰수하라 나라 망신이다,0.0,1NoOC,wowg****,0.0,0.0,2023-05-13T01:11:50+0900,791587287866540225,791587287866540225,100,2023-05-19
3,https://n.news.naver.com/mnews/article/comment...,민주당아 제발 참사팔이 하지 마라. 고인과 유족에 대한 모욕이다.,0.0,5uu6R,gsc3****,0.0,0.0,2023-05-12T23:40:51+0900,791581425856938227,791581425856938227,100,2023-05-19
4,https://n.news.naver.com/mnews/article/comment...,"구급차 자기 태우게 해 구조시간 지연시킨 신현영, 코인질이나 한 김남국 모두 더블빽...",0.0,4BrDw,li32****,1.0,0.0,2023-05-12T20:50:37+0900,791570458473398523,791570458473398523,100,2023-05-19
...,...,...,...,...,...,...,...,...,...,...,...,...
130960,https://n.news.naver.com/mnews/article/comment...,"40년이 지나도록 풀리지 않는,광주 시민 누구도 거론 하지 않는,5.18의 핵심은 ...",0.0,2veLj,fact****,4.0,1.0,2023-05-16T16:56:05+0900,791926434338701387,791926434338701387,100,2023-05-19
130961,https://n.news.naver.com/mnews/article/comment...,5.18은 당시 구금된 김대중 대통령 후보를 지지하기 위해 일어난 '전두한 장군과의...,0.0,1nZ8x,jhob****,1.0,1.0,2023-05-16T13:00:14+0900,791911239935590517,791911239935590517,100,2023-05-19
130962,https://n.news.naver.com/mnews/article/comment...,전라도당의 야밤 대자보''....//'미국산 소고기 먹지마라'며 선동..ㅡ&gt;미...,0.0,1nZ8x,jhob****,0.0,1.0,2023-05-16T12:58:52+0900,791911151418999129,791911151418999129,100,2023-05-19
130963,https://n.news.naver.com/mnews/article/comment...,친일정권 몰아내고 민주화세대가 정권을 잡아야한다!! 일제에 저항하고 중화문명을 수호...,0.0,3KtuF,toas****,2.0,4.0,2023-05-16T11:58:34+0900,791907266268758288,791907266268758288,100,2023-05-19


In [1]:
df.to_csv(f'./data/{str(now)}_naver_daily_news.csv', index=False)

NameError: name 'df' is not defined